# **ЗАДАНИЕ:**
1. Создайте таблицу movies с полями movies_type, director, year_of_issue, length_in_minutes, rate.

2. Сделайте таблицы для горизонтального партицирования по году выпуска (до 1990, 1990 -2000, 2000- 2010, 2010-2020, после 2020).

3. Сделайте таблицы для горизонтального партицирования по длине фильма (до 40 минута, от 40 до 90 минут, от 90 до 130 минут, более 130 минут).

4. Сделайте таблицы для горизонтального партицирования по рейтингу фильма (ниже 5, от 5 до 8, от 8до 10).

5. Создайте правила добавления данных для каждой таблицы.

6. Добавьте фильмы так, чтобы в каждой таблице было не менее 3 фильмов.

7. Добавьте пару фильмов с рейтингом выше 10.

8. Сделайте выбор из всех таблиц, в том числе из основной.

9. Сделайте выбор только из основной таблицы.

In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=6c62295e607df2b84c9106a71434f69db4067b9aa330bac06fbc48b2c19228f1
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
! pip install MySQL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.2.1-cp310-cp310-linux_x86_64.whl size=123923 sha256=48d5f0bd3e59b602106d28a6b0e6c3a9bf06320d10f1b568ca5a501c1b0585f7
  Stored in directory: /root/.cache/pip/wheels/93/84/49/615c5604001ebe3bb11c45628f22102cf563d6766a43a0e7a1
Successfully built mysqlclient


In [ ]:
! pip install sqlalchemy

In [ ]:
import pyspark, time, platform, sys, os
from datetime import datetime
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import col, lit, current_timestamp
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import inspect, create_engine, select, column, text as sql_text
from pandas.io import sql
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from pyspark.sql.functions import col, lit

In [ ]:
con = create_engine("mysql:///:memory:", echo=True)
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
columns = ["id","movies_type", "director", "title", "year_of_issue", "length_in_minutes", "rate"]

data_main = [(1, 'detective', 'Igor Maslennikov', 'The Adventures of Sherlock Holmes and Doctor Watson: The Hound of the Baskervilles', 1981, 154, 8.6),
        (2, 'comedy', 'Billy Wilder', 'Some Like It Hot', 1959, 119, 8.5),
        (3, 'drama', 'Frank Darabont','The Green Mile', 1999, 189, 9.1),
        (4, 'romantic comedy', 'Richard Curtis', 'Love Actually',  2003, 129, 7.9),
        (5, 'comedy', 'Damir Miftakhov','Christmas trees 10', 2023, 90, 4.1),
        (6, 'comedy', 'Ron Howard','How the Grinch Stole Christmas', 2000, 104, 6.8),
        (7, 'drama', 'Frank Darabont','The Green Mile', 1999, 189, 9.1),
        (8, 'cartoon', 'Vladimir Toropchin','Ivan Tsarevich and the Grey Wolf', 2011, 86, 6.7),
        (9, 'drama', 'Olivier Nakache','Intouchables', 2011, 112, 8.8)]

df = spark.createDataFrame(data_main, columns)
df.show()

#Добавьте фильмы
df2 = spark.createDataFrame([[10, 'comedy', 'Leonid Gaidai', 'Самогонщики', 1962, 19, 8.3]],
 ["id","movies_type", "director", "title", "year_of_issue", "length_in_minutes", "rate"])
df = df.unionByName(df2)
df.show()
print("*"*20)
# Сделайте таблицы для горизонтального партицирования по году выпуска (до 1990, 1990 -2000, 2000- 2010, 2010-2020, после 2020).
df_less1990 = df.filter( (col('year_of_issue') < lit(1990)) )
df_less1990.show()
print("*"*20)
df_1990_2000 = df.filter( (col('year_of_issue') >= lit(1990)) & (col('year_of_issue') < lit(2000)))
df_1990_2000.show()
print("*"*20)
df_2000_2010 = df.filter( (col('year_of_issue') >= lit(2000)) & (col('year_of_issue') < lit(2010)))
df_2000_2010.show()
print("*"*20)
df.filter( (col('year_of_issue') >= lit(2010)) & (col('year_of_issue') < lit(2020))).show()
print("*"*20)
df.filter( (col('year_of_issue') >= lit(2020)) ).show()
print("*"*20)

# Сделайте таблицы для горизонтального партицирования по длине фильма (до 40 минута, от 40 до 90 минут, от 90 до 130 минут, более 130 минут)
df.filter( (col('length_in_minutes') < lit(40)) ).show()
print("*"*20)
df.filter( (col('length_in_minutes') >= lit(40)) & (col('length_in_minutes') < lit(90))).show()
print("*"*20)
df.filter( (col('length_in_minutes') >= lit(90)) & (col('length_in_minutes') < lit(130))).show()
print("*"*20)
df.filter( (col('length_in_minutes') >= lit(130)) ).show()
print("*"*20)

# Сделайте таблицы для горизонтального партицирования по рейтингу фильма (ниже 5, от 5 до 8, от 8до 10)
df.filter( (col('rate') < lit(5)) ).show()
print("*"*20)
df.filter( (col('rate') >= lit(5)) & (col('rate') < lit(8))).show()
print("*"*20)
df.filter( (col('rate') >= lit(8)) & (col('rate') < lit(10))).show()
print("*"*20)



# Сделайте выбор из всех таблиц, в том числе из основной.
df.select(["id", "title", "year_of_issue", "rate"]).show(5)
print("*"*20)
(df_2000_2010.select(["title", "year_of_issue", "rate"])).filter((col('length_in_minutes') >= lit(110)) ).show()
print("*"*20)


t0 = time.time()
spark.stop()
t1 = time.time()
print('finished'), time.strftime('%H:%M:%S', time.gmtime(round(t1-t0)))

+---+---------------+------------------+--------------------+-------------+-----------------+----+
| id|    movies_type|          director|               title|year_of_issue|length_in_minutes|rate|
+---+---------------+------------------+--------------------+-------------+-----------------+----+
|  1|      detective|  Igor Maslennikov|The Adventures of...|         1981|              154| 8.6|
|  2|         comedy|      Billy Wilder|    Some Like It Hot|         1959|              119| 8.5|
|  3|          drama|    Frank Darabont|      The Green Mile|         1999|              189| 9.1|
|  4|romantic comedy|    Richard Curtis|       Love Actually|         2003|              129| 7.9|
|  5|         comedy|   Damir Miftakhov|  Christmas trees 10|         2023|               90| 4.1|
|  6|         comedy|        Ron Howard|How the Grinch St...|         2000|              104| 6.8|
|  7|          drama|    Frank Darabont|      The Green Mile|         1999|              189| 9.1|
|  8|     

(None, '00:00:01')